In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import json

WEIGHTS = {
    "population": 0.5,
    "green_deficit": 0.3,
    "temperature": 0.2
}
def load_data(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
    elif file_path.endswith(".geojson"):
        df = pd.read_json(file_path)
    else:
        raise ValueError("Unsupported file format. Use CSV or GeoJSON.")
    df = df.dropna(subset=["zone_id", "avg_ndvi", "population_density"])
    df = df.fillna({"surface_temp": df["surface_temp"].mean() if "surface_temp" in df else 0})
    return df
def normalize_features(df):
    scaler = MinMaxScaler()
    df["normalized_ndvi"] = scaler.fit_transform(df[["avg_ndvi"]])
    df["green_deficit"] = 1 - df["normalized_ndvi"]
    df["population_score"] = scaler.fit_transform(df[["population_density"]])
    if "surface_temp" in df.columns:
        df["temperature_score"] = scaler.fit_transform(df[["surface_temp"]])
    else:
        df["temperature_score"] = 0

    return df
def compute_scores(df):
    df["priority_score"] = (
        WEIGHTS["population"] * df["population_score"] +
        WEIGHTS["green_deficit"] * df["green_deficit"] +
        WEIGHTS["temperature"] * df["temperature_score"]
    )
    return df
def classify_zones(df):
    df = df.sort_values("priority_score", ascending=False).reset_index(drop=True)
    n = len(df)

    def assign_label(rank):
        if rank < 0.1 * n:
            return "Critical"
        elif rank < 0.3 * n:
            return "High"
        elif rank < 0.6 * n:
            return "Medium"
        else:
            return "Low"

    df["priority_level"] = [assign_label(i) for i in range(n)]
    return df
def add_explanations(df):
    explanations = []
    for _, row in df.iterrows():
        explanation = (
            f"Zone {row['zone_id']} is {row['priority_level']} because "
            f"NDVI={row['avg_ndvi']:.2f}, "
            f"Population={row['population_density']}, "
            f"Temp={row['surface_temp'] if 'surface_temp' in df.columns else 'N/A'}."
        )
        explanations.append(explanation)
    df["explanation"] = explanations
    return df
def export_geojson(df, output_path="scored_zones.geojson"):
    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "zone_id": row["zone_id"],
                "priority_score": row["priority_score"],
                "priority_level": row["priority_level"],
                "explanation": row["explanation"]
            }
        }
        features.append(feature)

    geojson = {"type": "FeatureCollection", "features": features}
    with open(output_path, "w") as f:
        json.dump(geojson, f, indent=2)

    print(f"Exported {output_path}")
def run_pipeline(file_path):
    df = load_data(file_path)
    df = normalize_features(df)
    df = compute_scores(df)
    df = classify_zones(df)
    df = add_explanations(df)
    export_geojson(df)
    return df
df_result = run_pipeline("zones.csv")
df_result.head()

Exported scored_zones.geojson


,zone_id,avg_ndvi,population_density,surface_temp,normalized_ndvi,green_deficit,population_score,temperature_score,priority_score,priority_level,explanation
0,A5,0.20,3000,40,0.166667,0.833333,1.00,1.0,0.95000,Critical,"Zone A5 is Critical because NDVI=0.20, Populat..."
1,A3,0.30,2200,37,0.375000,0.625000,0.68,0.7,0.66750,High,"Zone A3 is High because NDVI=0.30, Population=..."
2,A1,0.12,1500,35,0.000000,1.000000,0.40,0.5,0.60000,Medium,"Zone A1 is Medium because NDVI=0.12, Populatio..."
3,A2,0.45,800,32,0.687500,0.312500,0.12,0.2,0.19375,Low,"Zone A2 is Low because NDVI=0.45, Population=8..."
4,A4,0.60,500,30,1.000000,0.000000,0.00,0.0,0.00000,Low,"Zone A4 is Low because NDVI=0.60, Population=5..."
